In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
r_df = pd.read_csv('data\\Responses.csv')
r_df

In [ ]:
r_df.isnull().sum()

In [ ]:
r_df.drop(columns=['Timestamp','SIM Operator'],inplace=True)

In [ ]:
r_df.columns = ['gender','Partner','tenure','InternetService','StreamingTV','StreamingMovies','Contract','PaymentMethod','MonthlyCharges','Churn']
r_df

cpy_df = r_df.copy()

In [ ]:
for column in r_df.columns:
    print(f"{column}:{r_df[column].unique()}") 

In [ ]:
r_df.isnull().sum()

In [ ]:
def trinarizer(df,col):
    tri_lst = []
    for val in df[col]:
        if val == 'Yes':
            tri_lst.append(1)
        elif val == 'No':
            tri_lst.append(0)
        elif val == 'Monthly':
            tri_lst.append(1)
        elif val == 'Yearly':
            tri_lst.append(2)
        elif val == np.nan:
            tri_lst.append(np.nan)
            print(1)
    df[col] = tri_lst

In [ ]:
r_df['MonthlyCharges'] = r_df['MonthlyCharges'].fillna(r_df['MonthlyCharges'].mean())
r_df['tenure'] = r_df['tenure'].fillna(r_df['tenure'].mode()[0])
r_df['Partner'] = r_df['Partner'].fillna(r_df['Partner'].mode()[0])
r_df['Contract'] = r_df['Contract'].fillna(r_df['Contract'].mode()[0])
r_df['PaymentMethod'] = r_df['PaymentMethod'].fillna(r_df['PaymentMethod'].mode()[0])
r_df.isnull().sum()

In [ ]:
tot_charge = []
for tenure,charge,contr in zip(r_df['tenure'],r_df['MonthlyCharges'],r_df['Contract']):
    if contr == 'Yearly':
        res = tenure*charge/80
    elif contr == 'Monthly':
        res = tenure*12*charge/80
    tot_charge.append(res)
r_df['TotalCharges'] = tot_charge
r_df['MonthlyCharges'] = r_df['MonthlyCharges']/80

In [ ]:
cols = ['Partner','Contract','InternetService','StreamingTV','StreamingMovies','Churn']
for val in cols:
    trinarizer(r_df,val)
r_df

In [ ]:
billing = []
for method in r_df['PaymentMethod']:
    if method in ['UPI','Bank Transfer','Credit Card']:
        billing.append(1)
    elif method == 'Manual(from offline store)':
        billing.append(0)
r_df['PaperlessBilling'] = billing

In [ ]:
vals = []
for val in r_df['PaymentMethod']:
    if val == 'UPI':
        vals.append(1)
    elif val == 'Credit Card':
        vals.append(3)
    elif val == 'Bank Transfer':
        vals.append(2)
    elif val == 'Manual(from offline store)':
        vals.append(0)
r_df['PaymentMethod'] = vals
r_df

In [ ]:
x = [0]*len(r_df)
r_df['SeniorCitizen'] = x
r_df['tenure'] = r_df['tenure']*12
r_df

In [ ]:
r_df.groupby('gender').Churn.sum()

In [ ]:
for column in r_df.columns:
    print(f"{column}:{r_df[column].unique()}") 

In [ ]:
r_df.to_csv("data\\preprocessed_raw.csv",index= False)

In [ ]:
cpy_df.head()

In [ ]:
cpy_df = cpy_df.drop(columns=['gender','Partner','StreamingTV','StreamingMovies','MonthlyCharges'])
cpy_df.head()

In [ ]:
cpy_df.isnull().sum()

In [ ]:
cpy_df['Contract'] = cpy_df['Contract'].fillna(cpy_df['Contract'].mode()[0])
cpy_df['PaymentMethod'] = cpy_df['PaymentMethod'].fillna(cpy_df['PaymentMethod'].mode()[0])
cpy_df['tenure'] = cpy_df['tenure'].fillna(cpy_df['tenure'].mode()[0])

In [ ]:
import sim_encoding

use_df = sim_encoding.onehot_trans(col="PaymentMethod",df=cpy_df)
use_df = use_df.join(sim_encoding.onehot_trans(col="Contract",df=cpy_df))
use_df = use_df.join(sim_encoding.label_trans("InternetService",cpy_df))
use_df = use_df.join(sim_encoding.label_trans("Churn",cpy_df))
use_df['tenure'] = cpy_df['tenure']
use_df.head()

In [ ]:
use_df["PaymentMethod_Bank transfer (automatic)"] = use_df["PaymentMethod_Bank Transfer"]
use_df['Contract_Two year'] = use_df["Contract_Yearly"]
use_df['InternetService_Fiber optic'] = use_df['InternetService_enc']
use_df["Churn"] = use_df["Churn_enc"]
use_df['Contract_Month-to-month'] = use_df['Contract_Monthly']

In [ ]:
merge = pd.read_csv("features.csv")
cols = merge.columns

use_df = use_df.drop(columns=[col for col in use_df if col not in cols])
use_df['TechSupport_No'] = np.zeros(len(use_df))
use_df.head()

In [ ]:
merged_df = pd.concat([merge,use_df])
merged_df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled = scaler.fit_transform(merged_df[['tenure']])
merged_df['tenure'] = scaled

merged_df.head()

In [ ]:
merged_df.to_csv("collected_data.csv")